In [1]:
# Download the tiny shakespeare dataset
import requests
from pathlib import Path

data_dir = Path().resolve().parent / "data"

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

with open(f"{data_dir}/shakespeare.txt", "w", encoding="utf-8") as file:
    file.write(response.text)

In [2]:
# Read in the data to inspect it
with open(f"{data_dir}/shakespeare.txt", 'r', encoding="utf-8") as file:
    text = file.read()

In [3]:
# Lets look at the dataset
print(f"Length of the dataset: {len(text)}")
print('-' * 30)
print("First 200 characters:")
print('-' * 30)
print(text[:200])
print('-' * 30)

Length of the dataset: 1115394
------------------------------
First 200 characters:
------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you
------------------------------


In [4]:
# Further analyze the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)

print('-' * 30)
print("The dataset containts the following characters:")
print('-' * 30)
print(''.join(chars))
print('-' * 30)
print(f"Vocalubary size: {vocab_size}")
print('-' * 30)

------------------------------
The dataset containts the following characters:
------------------------------

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
------------------------------
Vocalubary size: 65
------------------------------


In [5]:
# Tokenize our charakters in a simple way to integers (instead of vectors)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print('-' * 30)
print(encode("hii there"))
print('-' * 30)
print(decode(encode("hii there")))
print('-' * 30)

------------------------------
[46, 47, 47, 1, 58, 46, 43, 56, 43]
------------------------------
hii there
------------------------------


In [6]:
# Encode the entire dataset and store it as a torch tensor
import torch
torch.cuda.is_available()
data = torch.tensor(encode(text), dtype=torch.long)

print('-' * 30)
print("Dataset as tensor:")
print('-' * 30)
print(data.shape, data.dtype)
print('-' * 30)
print(data[:200])

------------------------------
Dataset as tensor:
------------------------------
torch.Size([1115394]) torch.int64
------------------------------
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47

In [7]:
# Train / Val Split of the data
split_n = int(0.9 * len(data))

train_data = data[:split_n]
val_data = data[split_n:]

In [8]:
# Context Windows
block_size = 5
decode(train_data[:block_size+1].numpy())

'First '

In [9]:
# Show training examples
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context.numpy()}, the target is {target}")

when input is [18], the target is 47
when input is [18 47], the target is 56
when input is [18 47 56], the target is 57
when input is [18 47 56 57], the target is 58
when input is [18 47 56 57 58], the target is 1


In [10]:
torch.manual_seed(1337)
batch_size = 4 
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y 

xb, yb = get_batch('train')
print('-' * 30)
print('inputs:')
print(xb.shape)
print(xb)
print('-' * 30)
print('targets:')
print(yb.shape)
print(yb)
print('-' * 30)

------------------------------
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
------------------------------
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------


In [11]:
# Test output of training data and labels
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.numpy()}, the target is {target}")
    break

When input is [24], the target is 43
When input is [24 43], the target is 58
When input is [24 43 58], the target is 5
When input is [24 43 58  5], the target is 57
When input is [24 43 58  5 57], the target is 1
When input is [24 43 58  5 57  1], the target is 46
When input is [24 43 58  5 57  1 46], the target is 43
When input is [24 43 58  5 57  1 46 43], the target is 39


In [12]:
# Our Training Sample Batches
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # Becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss.item())

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) 4.878634929656982

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
from tqdm import tqdm

# Training Loop
for steps in tqdm(range(10000), desc="Training"):

    # sample a batch of data
    xb, xy = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, xy) # get loss and output
    optimizer.zero_grad()  # zero gradients
    loss.backward() # backprop
    optimizer.step() # compute gradients

print(loss.item())
    
    


Training: 100%|██████████| 10000/10000 [00:02<00:00, 3531.14it/s]

2.420325756072998


In [16]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


LUng:
j?
o f herere s digl y somm d fethal m,
KTIN fl tthane abe fls th un,
AD3XEr tidsethe n, or d GLGHAky!
G&paveelt pofotes dend tou:

AEVuing
SPKI' thickerenlaig, at wg wonofang dstobouThee CH:P! t DI fuf O my me he n:
Biset,Uinl, ke is tor, micuais s.
blt ind'thegandisins m.
W-ves: anyoroup meartold n mend? theral igr

CELK$z; ke me En s HOUpanalaf s yo'se, y h d hd t t.
Uhe dizk:
Clalesed wr


In [17]:
# mathematical trick in self-attention
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [18]:
# We want x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # [t,C]
        xbow[b,t] = torch.mean(xprev,0 )
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [19]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x # (B,T,T) @ (B,T,C) --> (B,T,C)
xbow2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [20]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
xbow3[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [30]:
# self attention
torch.manual_seed(1337)

B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)


tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape


torch.Size([4, 8, 16])

In [28]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [33]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [34]:
wei.var()

tensor(0.9957)